In [35]:
from numpy import loadtxt
import numpy as np
import glob
import os
import ast
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from scipy.misc import derivative
from sympy import symbols, solve

In [57]:
def calc_random_diff(genome_filename, mature_filename, canon_site):
    #load random simulation data into dictionary
    data_r = {}
    new_file_path = "Simulated Random Coverage/"+genome_filename+"_simulation/Canon Site "+canon_site
    for ID in glob.glob(os.path.join(new_file_path, '*.txt')):
           with open(os.path.join(os.getcwd(), ID), 'r') as f:
                data_r[int(os.path.basename(ID).split('/')[-1].strip(".tx"))] = (ast.literal_eval(f.read()))
                
    #sort dictionary
    keys_r = list(data_r.keys())
    keys_r.sort()
    data_r = {i: data_r[i] for i in keys_r}
    
    #generate means points (to plot line through boxplots)
    x = []
    y = []
    for i in data_r:
        x.append(i) 
        y.append(sum(data_r[i])/len(data_r[i]))
        
    #find the best fit using the generalized logistic function 
    def glf(x, a, b, c, d, g):
        return ( ( (a-d) / ( (1+( (x/c)** b )) **g) ) + d )
    popt, pcov = curve_fit(glf, x, y, p0=[0,1,np.quantile(x,.25),np.max(y),.4], maxfev=3000)
    
    def d_glf(x):
        return derivative(glf, x, dx = 1e-6, args = (popt))
    #plt.plot(x, [d_glf(i) for i in x])
    
    sol = -1
    for i in range(0,1600):
        if d_glf(i)<=1:
            sol = i
            break
            
    return (sol, d_glf(sol))

In [58]:
print(calc_random_diff('Zebrafish genes (GRCz11)','Zebrafish (Danio rerio)', 'B'))

(750, 0.999647454591468)


/var/folders/wd/wcjpwvbx6ql2slnp2p7cvn2m0000gn/T/ipykernel_64383/3485587334.py:23: RuntimeWarning: divide by zero encountered in power
  return ( ( (a-d) / ( (1+( (x/c)** b )) **g) ) + d )
/var/folders/wd/wcjpwvbx6ql2slnp2p7cvn2m0000gn/T/ipykernel_64383/3485587334.py:23: RuntimeWarning: invalid value encountered in power
  return ( ( (a-d) / ( (1+( (x/c)** b )) **g) ) + d )
/var/folders/wd/wcjpwvbx6ql2slnp2p7cvn2m0000gn/T/ipykernel_64383/3485587334.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return ( ( (a-d) / ( (1+( (x/c)** b )) **g) ) + d )
